In [1]:
import numpy as np
instances = "binpacking-66-132" # specify the instance class here

In [2]:
all_actions = np.load(f"../restricted_space/{instances}/action_space.npy")
all_scores  = np.load(f"../restricted_space/{instances}/action_scores.npy")
# one may change the path to the action space and action scores above

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
def check_action(action, actions):
    flag = 0
    for pre_action in actions:
        flag = 1
        for jj in range(17):
            if abs(pre_action[jj,0] - action[jj,0]) > 1e-9:
                flag = 0
                break
        if flag == 1:
            break
    return flag

def construct_smart_space(all_actions, all_scores, b, sas_size=30):
    # smart action space / mask upper bound action space
    smart_actions = [all_actions[0,:,:]]
    smart_actions_scores = [all_scores[0,:]]
    selected = np.zeros(all_actions.shape[0])
    selected[0] = 1
    for i in range(1,sas_size):
        index = None
        improv = -1
        cur_score = np.array(smart_actions_scores)[:i,:].max(axis=0).mean()
        for j in range(1,all_actions.shape[0]):
            if all_scores[j,:].mean() < b: # 0.3 default
                continue
            if selected[j] == 1:
                continue
            if check_action(all_actions[j,:,:], smart_actions) == True:
                selected[j] = 1
                continue
            new_score = np.array(smart_actions_scores[:i] + [all_scores[j,:]]).max(axis=0).mean()
            if new_score - cur_score > improv:
                index = j
                improv = new_score - cur_score
        if index == None:
            break
        smart_actions.append(all_actions[index,:,:])
        smart_actions_scores.append(all_scores[index,:])
    return smart_actions_scores

In [ ]:
our_size = 20
Bs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.525]
# adjust the size of the restricted space and the threshold above
means = []
ubs = []
all_stats = []
for b in Bs:
    smart_actions_scores = construct_smart_space(all_actions, all_scores, b, our_size * 3)
    means.append(np.array(smart_actions_scores[:our_size]).mean(axis=1).mean())
    ubs.append(np.array(smart_actions_scores[:our_size]).max(axis=0).mean())
    all_stats.append(smart_actions_scores)
print(means)
print(ubs)

In [ ]:
st = 0

sns.set_palette("colorblind")

plt.figure(figsize=(8, 4))

num_colors = 6

colorblind_palette = sns.color_palette("colorblind")

color = colorblind_palette[0] # rocket_palette[2]
plt.plot(Bs[st:], ubs[st:], label = "1st", color= color, marker='o')
plt.plot(Bs[st:], means[st:], label = "2nd", color= color,  linestyle = '--', marker='o')  # , linestyle = '--'


plt.legend()

fontsize = 15
# Set the plot title and labels
plt.xlabel('threshold $b$', fontsize=fontsize)
plt.ylabel('performace', fontsize=fontsize)

# Increase the legend font size
plt.legend(fontsize=fontsize)

# Set the font size of tick labels
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)

plt.grid(True)


plt.show()

In [ ]:
mean_bs = []
ub_bs = []
show_Bs = Bs[2:-1]
c_id = 0

num_colors = len([b for b in Bs if b in show_Bs])

colorblind_palette = sns.color_palette("colorblind")

rocket_palette = sns.color_palette("rocket", n_colors=5)[::-1]

mako_palette = sns.color_palette("mako", n_colors=num_colors)[::-1]
linestyle_cycler = ["-", "--", "-.", ":", "-", "--", "-."]
color_list = ["#045275", "#089099", "#7CCBA2", "#FCDE9C", "#FD746E", "#DC3977", "#7C1D6F"]

plt.figure(figsize=(8, 4))

for b_id, b in enumerate(Bs):
    if b not in show_Bs:
        continue
    mean_b = []
    ub_b = []
    for size_A in range(len(all_stats[b_id])):
        mean_b.append(np.array(all_stats[b_id][:size_A+1]).mean(axis=0).mean())
        ub_b.append(np.array(all_stats[b_id][:size_A+1]).max(axis=0).mean())
        
    color = mako_palette[c_id] # colorblind_palette[2*c_id+1]
    
    c_id += 1
    linewidth = 1.6
    alpha=0.8
    ub_label = f"$b=${b}; 1st"
    mean_label = f"$b=${b}; 2nd"
        
    line, = plt.plot(range(1,1+len(all_stats[b_id])), ub_b, label = ub_label, 
                     color=color, alpha=alpha, linewidth=linewidth,  marker='.')  # 
    plt.plot(range(1,1+len(all_stats[b_id])), mean_b, label = mean_label, 
             color=color, linestyle='--', alpha=alpha, linewidth=linewidth,  marker='.')  #

    
    mean_bs.append(mean_b)
    ub_bs.append(ub_b)

plt.grid(True)

fontsize = 15
# Set the plot title and labels
plt.xlabel('subspace size $|A|$', fontsize=fontsize)
plt.ylabel('performance', fontsize=fontsize)

# Increase the legend font size
plt.legend(fontsize=fontsize, loc='center right', bbox_to_anchor=(1.43, 0.5), )

# Set the font size of tick labels
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xlim([0, 25])

plt.show()

In [45]:
final_size = 15
final_b = 0.4
# finalize the restricted subspace size and the threshold above
restricted_actions_space, _ = construct_smart_space(all_actions, all_scores, final_b, final_size)
np.save(f"../restricted_space/{instances}/restricted_actions_space.npy", np.array(all_actions)[:final_size,:,:])
np.save(f"../restricted_space/{instances}/inst_agnostic_action.npy", np.array(all_actions)[0,:,:])
# one may change the save directory above